# Concepts
Beofre we dive into the details of `torchfsm`, let's introduce some key concepts in `torchfsm`.

## Operator

`Operator` is the key concept in `torchfsm`. It represents a **spatial** operation of a variable. For example, the gradient operator is an operator that computes the gradient of a variable. The divergence operator is an operator that computes the divergence of a variable. The laplacian operator is an operator that computes the laplacian of a variable. 

In `torchfsm`, there are three types of operator classes: `LinearOperator`, `NonlinearOperator` and `Operator`. `LinearOperator` and `NonlinearOperator` are classes represents linear and non-linear terms in the context of Fourier Spectal method (See our [Theory Introduction](../theory/#linear-terms)). `Operator` class is a combined operator that both contains linear and nonlinear terms.

All `Operator` class supports basic linear algebra operations, such as addition, subtraction and multiplication. Thanks to the linear feature of Fourier transformation, we can construct a PDE with different `Operator`s efficiently. For example, we can construct a [Burgers equation](https://en.wikipedia.org/wiki/Burgers%27_equation), $\frac{\partial \mathbf{u}}{\partial t}=-\mathbf{u} \cdot \nabla \mathbf{u} + \nu \nabla^2 \mathbf{u}$, as follows:

In [ ]:
from torchfsm.operator import Operator, Convection, Laplacian

def Burgers(nu:float) -> Operator:
    return nu*Laplacian()-Convection()
burgers=Burgers(0.01)

There are three important functions for `Operator` classes:

* `__call__`: Directly call the operator as a function will return the value of the operator applied to the input. This can be used to evaluate complex spatial derivatives.
* `integrate`: Integrate the operator in the time dimension. This is mainly used to solve time-dependent PDEs. 
* `solve`: Solve the linear operation with a given right-hand side. This only works for `LinearOperator` class and can be used to solve time-independent problems, such as Poisson equation.

In the later sections, we will further explain how to use these functions one by one.

All avaliable operators in `torchfsm` are summarized in the [cheet sheet](../cheetsheet/#operators).

## Mesh

In Fourier spectral method, all the boundary conditions are periodic. Meanwhile, any obstacles inside of the domain are not allowed. Therefore, we can use a very simple `Mesh` to represent the spatial domain. `Mesh` is another key concept in `torchfsm`. It represents the spatial domain of a PDE. It contains the information of the domain size and grid spacing.

All basic operations of an `Operator` class need a "mesh" as input. The mesh can be one of the following types:

* `mesh_info (Sequence[tuple[float,float,int]])`: `mesh_info` is a squence of tuple. Each tuple contains three elements: the start point of the domain, the end point of the domain, and the number of grid points in the domain. For example, `mesh_info=[(0,1,64),(0,1,64)]` represents a 2D domain with size $[0,1]\times[0,1]$ and grid spacing $dx=dy=1/64$.

* `MeshGrid`: `MeshGrid` is an interable class that reprents a mesh grid. This class is particularly useful for generating the initial condition. The length of the class is the number of mesh dimensions. The attribute x, y, z are the mesh grid for the first three dimension. You can also access the mesh grid for other dimension by indexing the object.E.g., `mesh_grid[0]` is the mesh grid for the first dimension, equivalent to x.There is no limit for the number of dimension. Assume that the number of points in each dimension is $n_1, n_2, n_3, \cdots, n_k$, the mesh grid will be of shape $(n_1,n_2,n_3,...,n_k)$. While for the attribute x, y, z, the shape will be $(n_1)$, $(n_2)$, $(n_3)$ respectively. `MeshGrid` can be initialized with a `mesh_info` with additional keyword to speicy the device and the data type of the mesh grid.

*  `FourierMesh`: `FourierMesh` is a class contains the fft frequency information and basic deritivate operators for the Fourier spectral method. This class is used inside of an Operator class. That is, all other meshes will be converted to `FourierMesh` inside of an Operator class. So if you want to save the meory for multiple operators, you should directly pass a `FourierMesh` to the operator, otherwise different operators will have different `FourierMesh` objects but with the same information. `FourierMesh` can be initialized with a `mesh_info` or a `MeshGrid` object.

## Data shape

In `torchfsm` is designed for deep learning, all the data are manipulated in the form of PyTorch Tensors. Thus, the data shape also follows the PyTorch convention. There are two types of data shapes in `torchfsm`:

* $[B,C,H,W,\cdots]$: This is the most common data shape in `torchfsm`. $B$ is the batch size, $C$ is the number of channels, $H,W,\cdots$ are spatial dimensions. For example, 32 2D vector fields on a 2D 64$\times$64 mesh can be represented as $[32,1,64,64]$. All the input of the `Operator` class should be in this shape.
* $[B,T,C,H,W,\cdots]$: This data shape is used to store trajectory data with the additional trajectory dimension $T$. $T$ is the number of solution frames in the trajectory. For example, 32 2D vector fields on a 2D 64$\times$64 mesh with 100 time frames can be represented as $[32,100,2,64,64]$. This data shape is used as an output when solving time-dependent PDEs.